In [1]:
!pip install pandas_datareader


## Factor Analysis of JPMorgan Using CAPM and Fama-French Models

- What explains JPM’s returns? Is it just the market? Or other risk factors?

#### Step 1: Download JPM Monthly Returns

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

# Download daily data
jpm = yf.download("JPM", start="2015-01-01", auto_adjust=True)[["Close"]]

# Convert to monthly returns
jpm = jpm.resample("M").last().pct_change().dropna()

# Rename column
jpm.columns = ["Return"]

jpm.head()

[*********************100%***********************]  1 of 1 completed


,Return
Date,
2015-02-28,0.126885
2015-03-31,-0.011423
2015-04-30,0.051179
2015-05-31,0.039836
2015-06-30,0.030100


In [4]:
jpm.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 133 entries, 2015-02-28 to 2026-02-28
Freq: ME
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Return  133 non-null    float64
dtypes: float64(1)
memory usage: 2.1 KB


#### Step 2: Download Fama-French Factors

In [3]:
ff = web.DataReader("F-F_Research_Data_Factors", "famafrench", start="2015-01-01")[0]

# Convert from percent to decimal
ff = ff / 100

# Convert index to timestamp (month end to match resample("M"))
ff.index = ff.index.to_timestamp("M")

ff.head()

# Mkt-RF: Market excess return. If this is high, market did well.
# SMB: Small-cap returns minus large-cap returns. If SMB is positive, small stocks outperform big stocks.
# HML: Value stocks minus growth stocks. if HML is positive, value stocks outperform growth.
# RF: Risk-free rate (1-month T-bill). Used to compute excess return.

,Mkt-RF,SMB,HML,RF
Date,,,,
2015-01-31,-0.0310,-0.0059,-0.0345,0.0
2015-02-28,0.0613,0.0061,-0.0179,0.0
2015-03-31,-0.0111,0.0305,-0.0038,0.0
2015-04-30,0.0059,-0.0299,0.0180,0.0
2015-05-31,0.0137,0.0095,-0.0111,0.0


##### check for total rows for Fama and French

In [5]:
print("First date:", ff.index.min())
print("Last date:", ff.index.max())
print("Total rows:", len(ff))

First date: 2015-01-31 00:00:00
Last date: 2025-12-31 00:00:00
Total rows: 132


#### Step 3: Merge datasets

In [6]:
df = jpm[['Return']].merge(ff, left_index=True, right_index=True)

print("Total rows:", len(df))
df.head()

Total rows: 131


,Return,Mkt-RF,SMB,HML,RF
Date,,,,,
2015-02-28,0.126885,0.0613,0.0061,-0.0179,0.0
2015-03-31,-0.011423,-0.0111,0.0305,-0.0038,0.0
2015-04-30,0.051179,0.0059,-0.0299,0.0180,0.0
2015-05-31,0.039836,0.0137,0.0095,-0.0111,0.0
2015-06-30,0.030100,-0.0152,0.0294,-0.0082,0.0


#### Step 4: Compute excess return

In [7]:
df['Excess_Return'] = df['Return'] - df['RF']
df.head()

# in CAPM and Fama-French we do not model raw returns, we model excess resturns
# Ex: JPM returns 8% and Risk-free rate is 3%. You could have earned 3% with zero risk. so the true reward for taking risk is:
# 8% - 3% = 5%

,Return,Mkt-RF,SMB,HML,RF,Excess_Return
Date,,,,,,
2015-02-28,0.126885,0.0613,0.0061,-0.0179,0.0,0.126885
2015-03-31,-0.011423,-0.0111,0.0305,-0.0038,0.0,-0.011423
2015-04-30,0.051179,0.0059,-0.0299,0.0180,0.0,0.051179
2015-05-31,0.039836,0.0137,0.0095,-0.0111,0.0,0.039836
2015-06-30,0.030100,-0.0152,0.0294,-0.0082,0.0,0.030100


#### Step 5: CAPM REGRESSION

In [8]:
import statsmodels.api as sm

X_capm = df['Mkt-RF']
X_capm = sm.add_constant(X_capm)

y = df['Excess_Return']

capm_model = sm.OLS(y, X_capm).fit()
print(capm_model.summary())

# CAPM is all about risk premium: Investors are only rewarded for taking systematic risk.
# When the market earns extra return above risk-free, how much does JPM earn above risk-free.
# R2= 0.545: 54.5% of JPM’s excess return variation is explained by market movements.
# Beta (Mkt-RF)= 1.1297: If market excess return increases by 1%, JPM excess return increases by 1.13%.
# Alpha (const) = 0.0052: 0.52% per month abnormal return. alpha is NOT statistically significant.

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     154.5
Date:                Thu, 19 Feb 2026   Prob (F-statistic):           8.32e-24
Time:                        23:41:56   Log-Likelihood:                 217.14
No. Observations:                 131   AIC:                            -430.3
Df Residuals:                     129   BIC:                            -424.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0052      0.004      1.237      0.2

#### Step 6: FAMA-FRENCH REGRESSION

In [9]:
X_ff = df[['Mkt-RF', 'SMB', 'HML']]
X_ff = sm.add_constant(X_ff)

ff_model = sm.OLS(y, X_ff).fit()
print(ff_model.summary())

# we are now explaining JPM market risk, size exposure and value exposure
# Now 73.4% of JPM’s excess return variation is explained. size + value factors add meaningful explanatory power.
# Beta (Mkt-RF)= 1.12: If market excess return increases by 1%, JPM excess return increases by 1.12%.
# Alpha (const = 0.0061): Alpha got slightly larger and closer to significance.
# SMB = 0.0586 but p-value > 0.05: JPM does NOT behave like a small-cap stock.
# HML = 0.7964 and p-value < 0.05: JPM behaves strongly like a VALUE stock.

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     117.0
Date:                Thu, 19 Feb 2026   Prob (F-statistic):           2.18e-36
Time:                        23:42:32   Log-Likelihood:                 252.38
No. Observations:                 131   AIC:                            -496.8
Df Residuals:                     127   BIC:                            -485.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0061      0.003      1.875      0.0

- JPM has a beta greater than 1 (~1.12), meaning it is more volatile than the overall market.
- Under CAPM, the market factor alone explains about 54% of JPM’s excess returns (R² = 0.545).
- When adding size (SMB) and value (HML) factors in the Fama-French model, explanatory power increases significantly to 73% (R² = 0.734).
- The market factor (Mkt-RF) is highly statistically significant in both models, confirming that market movements strongly drive JPM returns.
- The value factor (HML) is statistically significant, suggesting JPM behaves more like a value stock.
- The size factor (SMB) is not statistically significant, consistent with JPM being a large-cap company.
- The intercept (alpha) is not statistically significant, indicating JPM does not generate abnormal returns after accounting for risk factors.
- Overall, the Fama-French model provides a better explanation of JPM’s returns than CAPM, highlighting the importance of multi-factor risk models.